# IFC Visualization in Plotly



## Installation Check

Make sure to have plotly and ifcopenshell installed properly. 

In [ ]:
import ifcopenshell
import ifcopenshell.geom

# Check if OCC is available
try:
    settings = ifcopenshell.geom.settings()
    print("OpenCascade is available.")
except Exception as e:
    print("OpenCascade is not available:", str(e))

## Visualization

In [ ]:
import ifcopenshell
import ifcopenshell.geom
import plotly.graph_objects as go
import numpy as np

# Load the IFC file
ifc_file = ifcopenshell.open("data\Duplex_A_20110907.ifc")

# Set up the geometry settings
settings = ifcopenshell.geom.settings()
settings.USE_PYTHON_APPS = True

# Extract all walls from the IFC model
walls = ifc_file.by_type("IfcWindow")

# Lists to hold the wall vertices
x, y, z = [], [], []
i, j, k = [], [], []  # Triangle indices for mesh

# Extract geometry for each wall
for wall in walls:
    # Create a shape representation using OpenCascade
    shape = ifcopenshell.geom.create_shape(settings, wall)
    
    faces = shape.geometry.faces  # Indices of vertices per triangle face
    verts = shape.geometry.verts  # Flattened list of vertex coordinates

    # Reshape verts to separate x, y, z coordinates
    num_vertices = len(verts) // 3
    verts = np.array(verts).reshape((num_vertices, 3))
    
    # Append vertices to the lists
    x.extend(verts[:, 0])
    y.extend(verts[:, 1])
    z.extend(verts[:, 2])
    
    # Append face indices for triangles
    for idx in range(0, len(faces), 3):
        i.append(faces[idx])
        j.append(faces[idx + 1])
        k.append(faces[idx + 2])

# Create a 3D mesh plot
mesh = go.Mesh3d(
    x=x,
    y=y,
    z=z,
    i=i,
    j=j,
    k=k,
    opacity=0.5,
    color='blue'
)

# Create the figure and add the mesh
fig = go.Figure(data=[mesh])

# Set the layout for the plot
fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    ),
    title='Wall Geometries from IFC Model',
)

# Show the plot
fig.show()